# ETL Maps union
Tomo el archivo de Harlan de metadato y lo uno con las reviews pero agrego fecha


In [16]:
# Cargo librerías
import pandas as pd
import numpy as np

In [17]:
maps = pd.read_parquet("./Maps/Resultado_unionV2/reviews-metadata-union.parquet")
tam_orig = maps.shape
print(tam_orig)

(520765, 16)


In [18]:
maps.head(2)

,user_id,name_x,time,rating,text,gmap_id,estado,name_y,address,description,latitude,longitude,category,avg_rating,num_of_reviews,category_string
0,1.178177e+20,tj paper,1630598450114,1,Not putting a filthy piece of fabric over my f...,0x880f97fc018a1333:0xf34461f47468c83,Illinois,Binny's Beverage Depot,"Binny's Beverage Depot, 290 Hawthorn Village C...","Longtime supplier for beer, wine & spirits, pl...",42.241681,-87.956607,"[Liquor store, Beer store, Wine store]",4.4,26,"Liquor store, Beer store, Wine store"
1,1.178177e+20,tj paper,1630598450114,1,Not putting a filthy piece of fabric over my f...,0x880f97fc018a1333:0xf34461f47468c83,Illinois,Binny's Beverage Depot,"Binny's Beverage Depot, 290 Hawthorn Village C...","Longtime supplier for beer, wine & spirits, pl...",42.241681,-87.956607,"[Liquor store, Beer store, Wine store]",4.4,26,"Liquor store, Beer store, Wine store"


In [19]:
maps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520765 entries, 0 to 520764
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   user_id          520765 non-null  float64
 1   name_x           520765 non-null  object 
 2   time             520765 non-null  int64  
 3   rating           520765 non-null  int64  
 4   text             298282 non-null  object 
 5   gmap_id          520765 non-null  object 
 6   estado           520765 non-null  object 
 7   name_y           520765 non-null  object 
 8   address          520566 non-null  object 
 9   description      309953 non-null  object 
 10  latitude         520765 non-null  float64
 11  longitude        520765 non-null  float64
 12  category         520765 non-null  object 
 13  avg_rating       520765 non-null  float64
 14  num_of_reviews   520765 non-null  int64  
 15  category_string  520765 non-null  object 
dtypes: float64(4), int64(3), object(9)
mem

In [20]:
for col in maps.columns:
    print("Los datos de la columna ", col, " son de tipo ", type(maps[col][0]))

Los datos de la columna  user_id  son de tipo  <class 'numpy.float64'>
Los datos de la columna  name_x  son de tipo  <class 'str'>
Los datos de la columna  time  son de tipo  <class 'numpy.int64'>
Los datos de la columna  rating  son de tipo  <class 'numpy.int64'>
Los datos de la columna  text  son de tipo  <class 'str'>
Los datos de la columna  gmap_id  son de tipo  <class 'str'>
Los datos de la columna  estado  son de tipo  <class 'str'>
Los datos de la columna  name_y  son de tipo  <class 'str'>
Los datos de la columna  address  son de tipo  <class 'str'>
Los datos de la columna  description  son de tipo  <class 'str'>
Los datos de la columna  latitude  son de tipo  <class 'numpy.float64'>
Los datos de la columna  longitude  son de tipo  <class 'numpy.float64'>
Los datos de la columna  category  son de tipo  <class 'numpy.ndarray'>
Los datos de la columna  avg_rating  son de tipo  <class 'numpy.float64'>
Los datos de la columna  num_of_reviews  son de tipo  <class 'numpy.int64'>
Los

In [21]:
# Cambio de formato de time para calcular duplicados
maps_copy = maps.copy()
maps_copy['time'] = pd.to_datetime(maps['time'], unit= 'ms')

In [22]:
#Eliminacion de duplicados
maps_copy.drop_duplicates(subset= ['name_x', 'time', 'text'], keep= 'first', inplace= True)


In [23]:
filas_duplicadas = tam_orig[0] - maps_copy.shape[0]
print("Filas duplicadas eliminadas: ", filas_duplicadas)

Filas duplicadas eliminadas:  19826


In [24]:
# Filas con reseñas nulas
maps_copy[maps_copy['text'].isna()]['user_id'].shape

(221202,)

In [25]:
# Creo un listado de usuarios que no escriben reseñas
user_null = maps_copy[maps_copy['text'].isna()]['user_id'].unique().tolist()
len(user_null)

208956

In [26]:
# Creo un dataset de los usuarios que no reseñan para evaluarlo
maps_usuarios_con_nulos = maps_copy[maps_copy['user_id'].isin(user_null)]

In [27]:
# Veo si los usuarios en ocasiones reseñan
maps_usuarios_con_nulos['text'].isnull().value_counts()

text
True     221202
False     11639
Name: count, dtype: int64

In [28]:
maps_usuarios_con_nulos.describe()

,user_id,time,rating,latitude,longitude,avg_rating,num_of_reviews
count,2.328410e+05,232841,232841.000000,232841.000000,232841.000000,232841.000000,232841.000000
mean,1.092441e+20,2019-03-13 22:55:32.737655552,4.452476,38.003696,-89.449918,4.409044,847.655177
min,1.000001e+20,1990-12-31 00:00:00,1.000000,19.642620,-159.463890,1.400000,5.000000
25%,1.046586e+20,2018-05-13 18:53:42.014000128,4.000000,34.175942,-95.169954,4.200000,74.000000
50%,1.092138e+20,2019-03-17 19:43:34.150000128,5.000000,39.163137,-84.397981,4.400000,264.000000
75%,1.138532e+20,2020-02-15 02:10:49.967000064,5.000000,41.704945,-78.870888,4.600000,1613.000000
max,1.184467e+20,2021-09-08 04:30:30.644000,5.000000,64.847681,-67.841240,5.000000,6882.000000
std,5.309423e+18,NaN,0.864413,5.275265,14.963867,0.289394,999.261585


In [29]:
maps_sin_nulos = maps_copy.dropna(subset= 'text')

In [30]:
# Guardo una copia
# maps_sin_nulos.to_parquet("ReviewMetadata_sin_nulos.parquet")

In [31]:
categorias = maps_sin_nulos['category_string'].to_list()

In [40]:
# Cantidad de repeticiones de categorias
categorias_unicas = list(set(categorias))
categorias_unicas

categorias_cuenta = {}

for elemento in categorias:
    if elemento in categorias_cuenta:
        categorias_cuenta[elemento] += 1
    else:
        categorias_cuenta[elemento] = 1

categorias_ordenada = dict(sorted(categorias_cuenta.items(), key= lambda item: item[1], reverse= True))

for key, value in categorias_ordenada.items():
    print(f"{key}: {value}")
                          



Italian restaurant, Caterer, Family restaurant, Gluten-free restaurant, Takeout restaurant, Seafood restaurant, Soup restaurant, Wine bar: 33200
Dollar store, Beer store, Clothing store, Discount store, Discount supermarket, School supply store, Tobacco shop, Toy store, Wine store: 16474
Grocery store, Cheese shop, Florist, Health food store, Natural goods store, Produce market, Supermarket, Wine store: 14735
Liquor store, Beer store, Wine store: 11627
Supermarket, Bakery, Beer store, Florist, Grocery store, Seafood market, Wine store: 7889
Wine store: 7878
Liquor store, Wine store: 7408
Grocery store, Beer store, Catering food and drink supplier, Flower delivery, Gourmet grocery store, Grocery delivery service, Produce market, Seafood market, Wine store: 6820
Winery, Tourist attraction: 6802
Winery: 6619
Italian restaurant, Caterer, Family restaurant, Gluten-free restaurant, Takeout Restaurant, Seafood restaurant, Soup restaurant, Wine bar: 6006
Seafood restaurant, American restaurant

In [17]:
cant_categorias = len(categorias)
categorias = list(set(categorias)) # Elimino duplicados

print("Cantidad de categorias: ", cant_categorias)
print("Categorias unicas: ", len(categorias))


Cantidad de categorias:  279737
Categorias unicas:  2696


In [18]:
categorias

['Deli, Beer store, Catering food and drink supplier, Family restaurant, Restaurant, Sandwich shop, Wine store',
 'Italian restaurant, Small plates restaurant, Southern Italian restaurant, Wine bar',
 'Restaurant, Deli, Wine store',
 'Wine store, Beer store, Liquor store, Shopping mall',
 'Liquor store, Cigar shop, Tobacco shop, Vaporizer store, Wine store',
 'Greek restaurant, Italian restaurant, Pizza restaurant, Wine bar',
 'Convenience store, Beer store, Dollar store, Drug store, Variety store, Wine store',
 'Wine cellar, Wine bar, Winery',
 'Convenience store, Bakery, Beer store, Dairy store, Donut shop, Ice cream shop, Liquor store, Wine store',
 'Convenience store, Beer store, Chocolate shop, Ice cream shop, Soft drinks shop, Wine store',
 'Florist, Brewing supply store, Wine store, Winemaking supply store',
 'Self-storage facility, Automobile storage facility, Boat storage facility, Moving and storage service, Moving supply store, RV storage facility, Storage facility, Warehous

# Limpiando reseñas

In [20]:
%pip install nltk
%pip install --upgrade numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [24]:

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')


def limpiar_texto(texto):
    # Eliminar puntuación y caracteres especiales
    texto = re.sub(r'[^\w\s]', '', texto)
    
    # Convertir a minúsculas
    texto = texto.lower()
    
    # Tokenización
    tokens = word_tokenize(texto)
    
    # Eliminar stop words en inglés
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Stemming (opcional)
    stemmer = SnowballStemmer("english")
    tokens = [stemmer.stem(word) for word in tokens]
    
    # Unir los tokens en un solo texto limpio
    texto_limpio = ' '.join(tokens)
    
    return texto_limpio

# Aplicar la función de limpieza a la columna "review_text"
maps_sin_nulos['clean_text'] = maps_sin_nulos['text'].apply(limpiar_texto)

# Mostrar el DataFrame resultante
print(maps_sin_nulos[['text', 'clean_text']])


[nltk_data] Downloading package punkt to /home/fermuba/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fermuba/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                     text  \
0       Not putting a filthy piece of fabric over my f...   
2       Nice and clean, can't wait till they get some ...   
4       Large, clean, neat, well organized.\nHelpful f...   
6       Vast selection great pricing for all kinds of ...   
8                                          Love binnys!!!   
...                                                   ...   
520734                                     Friendly staff   
520735                                      Great service   
520736                                  Cherry is da bomb   
520737                                        Great place   
520738                                Excellent selection   

                                               clean_text  
0       put filthi piec fabric face buy beer ill shop ...  
2          nice clean cant wait till get hard get bourbon  
4       larg clean neat well organ help friend staff s...  
6                    vast s

/tmp/ipykernel_4706/804321402.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maps_sin_nulos['clean_text'] = maps_sin_nulos['text'].apply(limpiar_texto)


In [25]:
maps_sin_nulos[['text', 'clean_text']]

,text,clean_text
0,Not putting a filthy piece of fabric over my f...,put filthi piec fabric face buy beer ill shop ...
2,"Nice and clean, can't wait till they get some ...",nice clean cant wait till get hard get bourbon
4,"Large, clean, neat, well organized.\nHelpful f...",larg clean neat well organ help friend staff s...
6,Vast selection great pricing for all kinds of ...,vast select great price kind alcohol
8,Love binnys!!!,love binni
...,...,...
520734,Friendly staff,friend staff
520735,Great service,great servic
520736,Cherry is da bomb,cherri da bomb
520737,Great place,great place


# Preparacion de archivos para el Mapa

In [46]:
# Winery, wineries, vineyard, wine tasting room
maps_sin_nulos.head()

,user_id,name_x,time,rating,text,gmap_id,estado,name_y,address,description,latitude,longitude,category,avg_rating,num_of_reviews,category_string
0,1.178177e+20,tj paper,2021-09-02 16:00:50.114,1,Not putting a filthy piece of fabric over my f...,0x880f97fc018a1333:0xf34461f47468c83,Illinois,Binny's Beverage Depot,"Binny's Beverage Depot, 290 Hawthorn Village C...","Longtime supplier for beer, wine & spirits, pl...",42.241681,-87.956607,"[Liquor store, Beer store, Wine store]",4.4,26,"Liquor store, Beer store, Wine store"
2,1.089516e+20,Alex Coyle,2021-08-29 01:32:12.937,5,"Nice and clean, can't wait till they get some ...",0x880f97fc018a1333:0xf34461f47468c83,Illinois,Binny's Beverage Depot,"Binny's Beverage Depot, 290 Hawthorn Village C...","Longtime supplier for beer, wine & spirits, pl...",42.241681,-87.956607,"[Liquor store, Beer store, Wine store]",4.4,26,"Liquor store, Beer store, Wine store"
4,1.008189e+20,Linda Simonpietri,2021-08-08 01:12:36.606,5,"Large, clean, neat, well organized.\nHelpful f...",0x880f97fc018a1333:0xf34461f47468c83,Illinois,Binny's Beverage Depot,"Binny's Beverage Depot, 290 Hawthorn Village C...","Longtime supplier for beer, wine & spirits, pl...",42.241681,-87.956607,"[Liquor store, Beer store, Wine store]",4.4,26,"Liquor store, Beer store, Wine store"
6,1.156822e+20,Irving Ubri,2021-08-16 13:12:46.442,5,Vast selection great pricing for all kinds of ...,0x880f97fc018a1333:0xf34461f47468c83,Illinois,Binny's Beverage Depot,"Binny's Beverage Depot, 290 Hawthorn Village C...","Longtime supplier for beer, wine & spirits, pl...",42.241681,-87.956607,"[Liquor store, Beer store, Wine store]",4.4,26,"Liquor store, Beer store, Wine store"
8,1.148888e+20,john sell,2021-08-17 15:06:24.259,5,Love binnys!!!,0x880f97fc018a1333:0xf34461f47468c83,Illinois,Binny's Beverage Depot,"Binny's Beverage Depot, 290 Hawthorn Village C...","Longtime supplier for beer, wine & spirits, pl...",42.241681,-87.956607,"[Liquor store, Beer store, Wine store]",4.4,26,"Liquor store, Beer store, Wine store"


In [47]:
maps_sin_nulos.columns

Index(['user_id', 'name_x', 'time', 'rating', 'text', 'gmap_id', 'estado',
       'name_y', 'address', 'description', 'latitude', 'longitude', 'category',
       'avg_rating', 'num_of_reviews', 'category_string'],
      dtype='object')

In [48]:
# Eliminar columnas innecesarias
columnas_eliminar = ['user_id', 'name_x', 'time', 'rating', 'text', 'gmap_id', 'estado',
                    'address', 'description', 'category', 'avg_rating', 'num_of_reviews']
maps_map = maps_sin_nulos.drop(columns= columnas_eliminar)
maps_map.head()

,name_y,latitude,longitude,category_string
0,Binny's Beverage Depot,42.241681,-87.956607,"Liquor store, Beer store, Wine store"
2,Binny's Beverage Depot,42.241681,-87.956607,"Liquor store, Beer store, Wine store"
4,Binny's Beverage Depot,42.241681,-87.956607,"Liquor store, Beer store, Wine store"
6,Binny's Beverage Depot,42.241681,-87.956607,"Liquor store, Beer store, Wine store"
8,Binny's Beverage Depot,42.241681,-87.956607,"Liquor store, Beer store, Wine store"


In [52]:
#Eliminar duplicados
maps_map.drop_duplicates(keep= 'first', inplace= True)

In [65]:
# Winery, Wine store, Wine bar, // Winery, Tourist attraction, Vineyard // Winery, Tourist attraction
map_winery = maps_map[maps_map['category_string'] == 'Winery']
map_wine_store = maps_map[maps_map['category_string'] == 'Wine store']
map_bar = maps_map[maps_map['category_string'] == 'Wine bar']
map_vineyard = maps_map[maps_map['category_string'] == 'Winery, Tourist attraction, Vineyard']
map_tourist = maps_map[maps_map['category_string'] == 'Winery, Tourist attraction']

print(map_winery.shape)
print(map_wine_store.shape)
print(map_bar.shape)
print(map_vineyard.shape)
print(map_tourist.shape)

(453, 4)
(506, 4)
(200, 4)
(51, 4)
(394, 4)


In [66]:
# Guardo los archivos para el mapa
map_winery.to_parquet("map_winery.parquet")
map_wine_store.to_parquet("map_wine_store.parquet")
map_bar.to_parquet("map_bar.parquet")
map_vineyard.to_parquet ("map_vineyard.parquet")
map_tourist.to_parquet("map_tourist.parquet")